In [23]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import defaultdict
import os
import pickle
import scipy.stats as st

from itertools import combinations
from scipy.stats import chi2

from code.algorithm_alpha import algorithm_alpha
from code.algorithm_t import algorithm_t
from scipy.special import factorial

sim = False

In [24]:
df = pd.read_csv('data/df_level2.csv')
df

,CNTSCHID,CNT,PRIVATESCH,STRATIO,SCHSIZE,sum_MATHbelow,mean_ESCS,mean_ESCS_std,Y_MATH,Y_BIN_MATH
0,800004,Albania,public,9.7619,205,11,-1.152958,-0.268234,5,1
1,800006,Albania,public,18.0000,315,15,-1.343747,-0.565595,5,1
2,800007,Albania,public,4.7368,45,2,-2.401967,-2.214922,4,1
3,800008,Albania,private,13.8125,221,6,-0.135900,1.316941,3,1
4,800009,Albania,public,16.9691,823,19,-0.560240,0.655569,2,0
...,...,...,...,...,...,...,...,...,...,...
12615,97500358,B-S-J-Z (China),public,8.0786,1131,0,0.828180,1.595021,0,0
12616,97500359,B-S-J-Z (China),public,11.3202,2298,0,-0.178994,0.251551,0,0
12617,97500360,B-S-J-Z (China),public,9.7932,2605,0,0.190169,0.743977,0,0
12618,97500361,B-S-J-Z (China),public,11.7941,2005,3,-0.929946,-0.750143,0,0


In [25]:
df['STRATIO_scaled'] = preprocessing.scale(df.STRATIO)
df['SCHSIZE_scaled'] = preprocessing.scale(df.SCHSIZE)
df['mean_ESCS_std_scaled'] = preprocessing.scale(df.mean_ESCS_std)

In [26]:
#df.CNT.unique()
df.groupby('CNT').count().reset_index()['CNT']
#df['Y_MATH'].hist()

0                  Albania
1                Argentina
2                Australia
3          B-S-J-Z (China)
4        Baku (Azerbaijan)
5                   Brazil
6        Brunei Darussalam
7                    Chile
8           Chinese Taipei
9                 Colombia
10              Costa Rica
11                 Denmark
12      Dominican Republic
13                 Estonia
14                  France
15                 Georgia
16                  Greece
17               Indonesia
18                 Ireland
19                  Israel
20                   Italy
21                   Japan
22                  Jordan
23              Kazakhstan
24                 Lebanon
25               Lithuania
26              Luxembourg
27                Malaysia
28                   Malta
29                  Mexico
30                 Moldova
31                 Morocco
32                  Panama
33                    Peru
34             Philippines
35                  Poland
36                Portugal
3

In [48]:
ran_var = False
ran_int = True
n_fix = 2
tol = 0.01  # set as 0.01, 0.05 or 0.10 if algorithm_alpha is used; 
            # otherwise use algorithm_t and tune the tolerance tol
model = 'P' # 'B' for Bernoulli and 'P' for Poisson

variab = 'SE'

In [49]:
N = df.CNT.nunique()

lengths = np.array(df.groupby('CNT').count().reset_index().iloc[:,1])

if model == 'B':
    y = list(df.groupby('CNT')['Y_BIN_MATH'].apply(np.array).values)
elif model == 'P':
    y = list(df.groupby('CNT')['Y_MATH'].apply(np.array).values)

fix = defaultdict(list)

if variab == "SE":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
    fix[1] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E
elif variab == "S":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
elif variab == "E":
    fix[0] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E

In [50]:
# with algorithm_alpha

knots, par, W, hess_ran, hess_fix, modeleval = algorithm_alpha(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N, t=None)

8.946569193878439e-175
8.946569193878439e-175
8.946569193878439e-175
2.1278940570280828e-91
1.575697025634062e-73
1.575697025634062e-73
1.575697025634062e-73
1.575697025634062e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5756994874986067e-73
1.5781143485697178e-73
1.5781143485697178e-73
1.5781143485697178e-73
1.5781143485697178e-73
1.5781143485697178e-73
3.42626906349496e-66
3.42626906349496e-66
3.42626906349496e-66
3.42626906349496e-66
3.42626906349496e-66
3.42626906349496e-66
3.42626906349496e-66
3.42626906349496e-66
3.4262690634951157e-66
3.4262690634951157e-66
3.4262690634951157e-66
3.4262690634951157e-66
3.4262690634951157e-66
3.426269

In [51]:
group = np.array([np.nan if np.sum(W[i,:])==0 else np.argmax(W[i,:]) for i in range(N)])

file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + str(variab) + '_' + str(tol) + '_' + str(model) + '_merging' + '.pickle'
f = open('output/case_study_results/' + file_name, 'wb')
pickle.dump([knots, par, W, hess_ran, hess_fix, group, modeleval],f)
f.close()

In [15]:
modeleval[0]

[array([-1.47585,  2.15392,  2.29333,  2.15055, -3.48443,  2.39224,
         1.19419,  2.11108, -2.82646,  2.23301,  1.16331, -2.20361,
         1.50076, -0.46921, -1.977  ,  0.33366, -2.77441,  2.20877,
         2.2016 , -2.85524,  1.59597, -3.36385, -0.8939 ,  0.32445,
         2.22802, -3.38929, -1.17126,  1.85154, -2.8473 , -3.39909,
         2.38108,  2.22793,  2.33327, -0.55273, -3.42275,  0.49859,
         1.38885, -3.50252, -0.272  , -2.43875, -1.60811, -0.92963,
        -3.50311, -0.53388, -0.9453 ,  2.3402 ,  2.23584, -3.48348,
        -1.4533 ,  0.30533]),
 array([-1.53512,  2.21397,  2.30425,  2.21152, -3.51266,  2.35151,
         1.17911,  2.18186, -2.8337 ,  2.26834,  1.15803, -2.1944 ,
         1.47248, -0.50277, -1.99705,  0.33097, -2.80908,  2.25245,
         2.24762, -2.84414,  1.58606, -3.481  , -0.95595,  0.31946,
         2.26512, -3.49114, -1.12533,  1.94349, -2.84139, -3.4944 ,
         2.34692,  2.26503,  2.32513, -0.55433, -3.50095,  0.48505,
         1.35945, 

In [18]:
[len(array) for array in modeleval[0]]

[50,
 50,
 50,
 50,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]

In [7]:
# with algorithm_t

# knots, par, W, hess_ran, hess_fix, modeleval = algorithm_t(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N, t=None)

1.5479635686021303e-47
1.3581950318750913e-39
1.3581950320540749e-39
7.733725946213973e-15
1.567274512505975e-13
1.567274512505975e-13
1.9349609228540352e-13
1.9349609228540352e-13
3.896589004078928e-08
3.896589004078928e-08
3.8965890040850286e-08
3.8965890040850286e-08
3.8965890040850286e-08
3.8965890040850286e-08
3.8965890040850286e-08
3.8965890040850286e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041216806e-08
3.8965890041484786e-08
3.8965890041484786e-08
2.724375678581383e-07
2.724375678581383e-07
2.724375678581387e-07
3.142717913396819e-07
3.142717913396819e-07
3.142717913396819e-07
3.1427496450353904e-07
3.1427496450353904e-07
3.1427496450353925e-07
3.1427496450353925e-07
3.142753793020257e-07
3.1555242574046683e-07
3.1555242574046683e-07
3.1555242574046683e-07
3.1555242574046683e-07
3.1555242574046683e-07


In [14]:
group = np.array([np.nan if np.sum(W[i,:])==0 else np.argmax(W[i,:]) for i in range(N)])

df.groupby('CNT').count().reset_index()['CNT']

0                  Albania
1                Argentina
2                Australia
3          B-S-J-Z (China)
4        Baku (Azerbaijan)
5                   Brazil
6        Brunei Darussalam
7                    Chile
8           Chinese Taipei
9                 Colombia
10              Costa Rica
11                 Denmark
12      Dominican Republic
13                 Estonia
14                  France
15                 Georgia
16                  Greece
17               Indonesia
18                 Ireland
19                  Israel
20                   Italy
21                   Japan
22                  Jordan
23              Kazakhstan
24                 Lebanon
25               Lithuania
26              Luxembourg
27                Malaysia
28                   Malta
29                  Mexico
30                 Moldova
31                 Morocco
32                  Panama
33                    Peru
34             Philippines
35                  Poland
36                Portugal
3

In [16]:
CI_length = np.array(st.norm.ppf(1 - tol / 2) * np.sqrt(1 / hess_ran))
pd.DataFrame({'lb': knots - CI_length.flatten(), 'ub': knots + CI_length.flatten()})

,lb,ub
0,-0.909725,-0.213219
1,-3.209331,-2.470325
2,1.782470,2.685237
3,-4.641562,-2.418206
4,0.209337,0.666958
5,-1.158109,-0.804384
6,-2.311653,-1.846308
7,0.934151,1.567852
8,0.837481,1.642361
9,-2.851021,-2.137573
